In [1]:
import cv2
import numpy as np
import time

In [8]:
def loadVideo():
    # Laden des Videos
    video_path = r'C:\Users\mtwar\Documents\MAS Data Science\CAS Machine Intelligence\Deep Learning\LNW\5_Video\WhatsApp Video 2025-05-12 at 11.45.15.mp4'
    #cap = cv2.VideoCapture(video_path)

    # Setze die Startposition auf 15 Sekunden (15000 Millisekunden)
    #start_time_ms = 15000
    #cap.set(cv2.CAP_PROP_POS_MSEC, start_time_ms)

    #video_path = r'C:\Users\mtwar\Documents\MAS Data Science\CAS Machine Intelligence\Deep Learning\LNW\5_Video\WhatsApp Video 2025-05-12 at 11.45.35.mp4'
    cap = cv2.VideoCapture(video_path)
    return cap

def loadVideoPath():
    # Laden des Videos
    video_path = r'C:\Users\mtwar\Documents\MAS Data Science\CAS Machine Intelligence\Deep Learning\LNW\5_Video\WhatsApp Video 2025-05-12 at 11.45.15.mp4'
    #cap = cv2.VideoCapture(video_path)

    # Setze die Startposition auf 15 Sekunden (15000 Millisekunden)
    #start_time_ms = 15000
    #cap.set(cv2.CAP_PROP_POS_MSEC, start_time_ms)

    #video_path = r'C:\Users\mtwar\Documents\MAS Data Science\CAS Machine Intelligence\Deep Learning\LNW\5_Video\WhatsApp Video 2025-05-12 at 11.45.35.mp4'
    return video_path


In [11]:
cap = loadVideo()

# Erstellen des Hintergrundsubtraktors
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=100, detectShadows=False)

# Original-Framerate des Videos
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_delay = int(1000 / original_fps)  # Verzögerung in Millisekunden

while cap.isOpened():
    start_time = time.time()

    ret, frame = cap.read()
    if not ret:
        break

    # Hintergrundsubtraktion anwenden
    fgmask = fgbg.apply(frame)

    # Morphologische Operationen zur Verbesserung des Masks
    kernel = np.ones((5, 5), np.uint8)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    # Konturen finden
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) < 500:  # Ignorieren kleiner Konturen
            continue

        # Umgebende Box zeichnen
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Vertikales Stapeln der Frames
    stacked_frame = np.vstack((frame, cv2.cvtColor(fgmask, cv2.COLOR_GRAY2BGR)))

    # Anzeigen des gestapelten Frames
    cv2.imshow('Stacked Frames', stacked_frame)

    # Berechnung der Zeit, die für die Verarbeitung benötigt wurde
    processing_time = int((time.time() - start_time) * 1000)

    # Warten, um die ursprüngliche Framerate beizubehalten
    if processing_time < frame_delay:
        cv2.waitKey(frame_delay - processing_time)
    else:
        # Wenn die Verarbeitung länger dauert als die Frame-Zeit, einfach zum nächsten Frame gehen
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [8]:
cap = loadVideo()

# Überprüfen, ob das Video erfolgreich geöffnet wurde
if not cap.isOpened():
    print("Fehler beim Öffnen der Videodatei")
    exit()

# Erstellen des Hintergrundsubtraktors
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=100, detectShadows=False)

# Original-Framerate des Videos
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_delay = int(1000 / original_fps)  # Verzögerung in Millisekunden

while cap.isOpened():
    start_time = time.time()

    ret, frame = cap.read()
    if not ret:
        break

    # Hintergrundsubtraktion anwenden
    fgmask = fgbg.apply(frame)

    # Morphologische Operationen zur Verbesserung der Maske
    kernel = np.ones((5, 5), np.uint8)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    # Konturen finden
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Zeichnen der Konturen auf dem Original-Frame
    for contour in contours:
        if cv2.contourArea(contour) < 500:  # Ignorieren kleiner Konturen
            continue

        # Umgebende Box zeichnen
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Anzeigen des Frames mit den erkannten Objekten
    cv2.imshow('Frame with Detected Objects', frame)

    # Berechnung der Zeit, die für die Verarbeitung benötigt wurde
    processing_time = int((time.time() - start_time) * 1000)

    # Warten, um die ursprüngliche Framerate beizubehalten
    if processing_time < frame_delay:
        cv2.waitKey(frame_delay - processing_time)
    else:
        # Wenn die Verarbeitung länger dauert als die Frame-Zeit, einfach zum nächsten Frame gehen
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Freigabe der Ressourcen
cap.release()
cv2.destroyAllWindows()


In [15]:
import cv2
import numpy as np
import time
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk

class VideoPlayer:
    def __init__(self, video_source):
        self.cap = cv2.VideoCapture(video_source)
        if not self.cap.isOpened():
            raise ValueError("Unable to open video source", video_source)

        self.history = 500
        self.varThreshold = 100

        self.fgbg = cv2.createBackgroundSubtractorMOG2(
            history=self.history, varThreshold=self.varThreshold, detectShadows=False)

        self.original_fps = self.cap.get(cv2.CAP_PROP_FPS)
        self.frame_delay = int(1000 / self.original_fps)

        self.root = tk.Tk()
        self.root.title("Video Player")

        self.canvas = tk.Canvas(self.root, width=self.cap.get(cv2.CAP_PROP_FRAME_WIDTH),
                                height=self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT)*2)
        self.canvas.pack()

         # History Slider
        self.history_label = tk.Label(self.root, text="History")
        self.history_label.pack()
        self.history_slider = ttk.Scale(self.root, from_=1, to=1000, orient=tk.HORIZONTAL,
                                        command=self.update_history)
        self.history_slider.set(self.history)
        self.history_slider.pack(fill=tk.X)

        # Threshold Slider
        self.threshold_label = tk.Label(self.root, text="Threshold")
        self.threshold_label.pack()
        self.varThreshold_slider = ttk.Scale(self.root, from_=1, to=200, orient=tk.HORIZONTAL,
                                             command=self.update_varThreshold)
        self.varThreshold_slider.set(self.varThreshold)
        self.varThreshold_slider.pack(fill=tk.X)

        self.update()

        self.root.mainloop()

    def update_history(self, value):
        self.history = int(float(value))
        self.update_bg_subtractor()

    def update_varThreshold(self, value):
        self.varThreshold = int(float(value))
        self.update_bg_subtractor()

    def update_bg_subtractor(self):
        self.fgbg = cv2.createBackgroundSubtractorMOG2(
            history=self.history, varThreshold=self.varThreshold, detectShadows=False)

    def update(self):
        ret, frame = self.cap.read()
        if ret:
            fgmask = self.fgbg.apply(frame)
            kernel = np.ones((5, 5), np.uint8)
            fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

            contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                if cv2.contourArea(contour) < 500:
                    continue
                (x, y, w, h) = cv2.boundingRect(contour)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            stacked_frame = np.vstack((frame, cv2.cvtColor(fgmask, cv2.COLOR_GRAY2BGR)))
            stacked_frame = cv2.cvtColor(stacked_frame, cv2.COLOR_BGR2RGB)
            img = ImageTk.PhotoImage(image=Image.fromarray(stacked_frame))

            self.canvas.create_image(0, 0, anchor=tk.NW, image=img)
            self.canvas.image = img
        else:
            self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            ret, frame = self.cap.read()

        self.root.after(self.frame_delay, self.update)

# Ersetze 'your_video.mp4' mit dem Pfad zu deinem Video
VideoPlayer(loadVideoPath())


In [9]:
import cv2
import numpy as np

cap = loadVideo()

#  Standardparameter
history = 500
varThreshold = 16
algo = 'MOG2'

# Hintergrundsubtraktor initialisieren
def initialize_subtractor(algo, history, varThreshold):
    if algo == 'MOG2':
        return cv2.createBackgroundSubtractorMOG2(history=history, varThreshold=varThreshold, detectShadows=False)
    elif algo == 'KNN':
        return cv2.createBackgroundSubtractorKNN(history=history, dist2Threshold=varThreshold, detectShadows=False)
    else:
        return cv2.createBackgroundSubtractorMOG2(history=history, nmixtures=3, backgroundRatio=0.7, noiseSigma=0)

fgbg = initialize_subtractor(algo, history, varThreshold)

# Callback-Funktion für die Schieberegler
def on_history_change(val):
    global history, fgbg
    history = val
    fgbg = initialize_subtractor(algo, history, varThreshold)

def on_threshold_change(val):
    global varThreshold, fgbg
    varThreshold = val
    fgbg = initialize_subtractor(algo, history, varThreshold)

# Hauptfenster erstellen
cv2.namedWindow('Frame')
cv2.createTrackbar('History', 'Frame', history, 1000, on_history_change)
cv2.createTrackbar('Threshold', 'Frame', varThreshold, 255, on_threshold_change)

while True:
    ret, frame = cap.read()
    if not ret:
        # Wenn das Video zu Ende ist, setzen Sie es zurück
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue

    # Hintergrundsubtraktion anwenden
    fgmask = fgbg.apply(frame)

    # Morphologische Operationen zur Verbesserung der Maske
    kernel = np.ones((5, 5), np.uint8)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    # Konturen finden
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Zeichnen der Konturen
    for contour in contours:
        if cv2.contourArea(contour) < 500:  # Ignorieren kleiner Konturen
            continue
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Anzeigen des Frames
    cv2.imshow('Frame', frame)

    # Tastatursteuerung für Algorithmusauswahl
    key = cv2.waitKey(1) & 0xFF
    if key == ord('1'):
        algo = 'KNN'
        fgbg = initialize_subtractor(algo, history, varThreshold)
    elif key == ord('2'):
        algo = 'MOG2'
        fgbg = initialize_subtractor(algo, history, varThreshold)
    elif key == ord('q'):
        break

# Aufräumen
cap.release()
cv2.destroyAllWindows()